In [0]:
import pandas as pd
from tqdm import tqdm

from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

In [0]:
import logging
logger = logging.getLogger('preselecting_wcomments_wrt_tox')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [0]:
# Connect to Google's BigQuery
project_id = "wikidetox-viz"
client = bigquery.Client(project=project_id)

In [0]:
def query(low_tox=0, high_tox=1, rand=.5, limit=1000, type="ADDITION", max_txt=1000):
    query = """
    SELECT * 
        FROM `wikiconv_v2.en_20180701_external`
        WHERE length(cleaned_content)>1 AND length(cleaned_content)<{max_txt} AND replyTo_id!="null" AND type="{typ}" AND toxicity>{low} AND toxicity<{high} AND RAND()<{ran}
        LIMIT {lim}
    """.format(low=low_tox, high=high_tox, ran=rand, lim=limit, typ=type, max_txt=max_txt)
    query_job = client.query(query)
    results = query_job.result()
    return results.to_dataframe()


In [0]:
def sample(bins=10):
    # Create random sample (initialization)
    table = query()
    table["toxic_zone"] = [-1] * table.shape[0]
    # Add samples following a toxicity-preselection
    for i in tqdm(range(bins)):
        preselected = query(low_tox=i / float(bins), high_tox=(i + 1) / float(bins))
        preselected["toxic_zone"] = [i] * preselected.shape[0]
        # Stack to initialized/updated table
        table = pd.concat([table, preselected], axis=0)
    # Shuffle
    table = table.sample(frac=1).reset_index(drop=True)
    return table


In [15]:
# run the sample code
table = sample()
table.to_csv("sample.11-toxic-zones.addition.csv", index=False)
table[:100].to_csv("sample.100posts.csv", index=False)


100%|██████████| 10/10 [02:17<00:00, 17.86s/it]


In [26]:
# ensure stratification of the sample
assert table.toxic_zone.count() == 11000
print (table.toxic_zone.value_counts())

 7    1000
-1    1000
 6    1000
 5    1000
 4    1000
 3    1000
 2    1000
 9    1000
 1    1000
 8    1000
 0    1000
Name: toxic_zone, dtype: int64
